In [1]:
import mysql.connector
import os

def get_db_connection():

    return mysql.connector.connect(
        host=os.getenv("MYSQL_HOST"),
        user=os.getenv("MYSQL_USER"),
        password=os.getenv("MYSQL_PASSWORD"),
        database=os.getenv("MYSQL_DATABASE")
    )

def data_into_db(data):

    connection = None
    try:
        # Establish database connection
        connection = get_db_connection()
        cursor = connection.cursor()

        insert_query = """
        INSERT INTO raw_data (
            ID, Delivery_person_Age, Delivery_person_Ratings, pickup_location_latitude, pickup_location_longitude,
            Delivery_location_latitude, Delivery_location_longitude, Order_Date, Time_Orderd, 
            Weatherconditions, Road_traffic_density, Vehicle_condition, Type_of_vehicle, 
            multiple_deliveries, City, Temperature, Traffic_Index, Time_taken
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        cursor.execute(insert_query, (
            data['ID'], data['Delivery_person_Age'], data['Delivery_person_Ratings'], 
            data['translogi_latitude'], data['translogi_longitude'], data['Delivery_location_latitude'], 
            data['Delivery_location_longitude'], data['Order_Date'], data['Time_Orderd'], 
            data['Weatherconditions'], data['Road_traffic_density'], data['Vehicle_condition'], 
            data['Type_of_vehicle'], data['multiple_deliveries'], data['City'], 
            data['Temperature'], data['Traffic_Index'], data['Time_taken']
        ))

        connection.commit()
        print("Record inserted successfully!")

    except mysql.connector.Error as e:
        print(f"Error inserting data into MySQL: {e}")
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("Database connection closed.")

In [3]:
# Sample data to insert
data = {
    "ID": "61916",
    "Delivery_person_Age": 37.0,
    "Delivery_person_Ratings": 4.9,
    "translogi_latitude": 22.745049,
    "translogi_longitude": 75.892471,
    "Delivery_location_latitude": 22.765049,
    "Delivery_location_longitude": 75.912471,
    "Order_Date": "2022-03-19",
    "Time_Orderd": "11:30:00",
    "Weatherconditions": "Sunny",
    "Road_traffic_density": "High",
    "Vehicle_condition": 2,
    "Type_of_vehicle": "motorcycle",
    "multiple_deliveries": 0.0,
    "City": "Urban",
    "Temperature": 29.0,
    "Traffic_Index": 1.2,
    "Time_taken": 24
}

# Insert the data
data_into_db(data)

Record inserted successfully!
Database connection closed.


In [9]:
import mysql.connector
import os

def get_db_connection():

    return mysql.connector.connect(
        host=os.getenv("MYSQL_HOST"),
        user=os.getenv("MYSQL_USER"),
        password=os.getenv("MYSQL_PASSWORD"),
        database=os.getenv("MYSQL_DATABASE")
    )


In [22]:

def save_route_directions_to_db(route_data):
    try:
        connection = get_db_connection()
        cursor = connection.cursor()
        
        # Convert instructions list to string
        instructions_str = '\n'.join(
            f"{i+1}. {instruction}" 
            for i, instruction in enumerate(route_data['directions']['instructions'])
        )
        
        insert_query = """
        INSERT INTO optimized_routes (
            pickup_lat, pickup_lng, delivery_lat, delivery_lng,
            total_distance, instructions
        ) VALUES (%s, %s, %s, %s, %s, %s)
        """
        
        values = (
            route_data['pickup']['lat'],
            route_data['pickup']['lng'], 
            route_data['delivery']['lat'],
            route_data['delivery']['lng'],
            route_data['directions']['distance'],
            instructions_str
        )
        
        cursor.execute(insert_query, values)
        connection.commit()
        
    except mysql.connector.Error as e:
        raise e
    finally:
        cursor.close()
        
test_route_data = {
    'pickup': {
        'lat': 12.9716,
        'lng': 77.5946
    },
    'delivery': {
        'lat': 13.0070,
        'lng': 77.5936
    },
    'directions': {
        'distance': 8500.7,
        'duration': 1234,
        'instructions': [
            'Start at Vidhana Soudha',
            'Head north on Dr Ambedkar Road',
            'Turn right onto Palace Road',
            'Continue onto Cunningham Road',
            'Turn left onto Miller Road',
            'Slight right onto Nandidurga Road',
            'Turn right onto Wheeler Road',
            'Continue onto Assaye Road',
            'Turn left onto Dickenson Road',
            'Arrive at destination'
        ],
        'path': [
            [12.9716, 77.5946],
            [12.9816, 77.5940],
            [12.9916, 77.5938],
            [13.0016, 77.5937],
            [13.0070, 77.5936]
        ]
    }
}

save_route_directions_to_db(test_route_data)